# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!type publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2019-01-01	Serials and Continuing Resources	K. A. Edwards and M. Leonard (Eds.), Assessment Strategies in Technical Services (pp.109-154)	This chapter is about the assessment of journals in academic libraries.	Calvert, K.R, and Jordan, W.J. (2019). "Serials and Continuing Resources." In K. A. Edwards and M. Leonard (Eds.), <i>Assessment Strategies in Technical Services</i>. (pp. 109-154). Chicago, IL: ALA Editions.	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf
2018-01-01	Collaborative leadership: cultivating an environment for success	Collaborative Librarianship	This paper is about collaboration and leadership.	Calvert, K.R. (2018). "Collaborative leadership: cultivating an environment for success." <i>Collaborative Librarianship</i>. 2(4).	paper-title-number-2	https://digitalcommons.du.edu/collaborativelibrarianship/vol10/iss2/4/ 
2015-01-01	Maximizing academic library collections: Measuring changes in use

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2019-01-01,Serials and Continuing Resources,"K. A. Edwards & M. Leonard (Eds.), Assessment ...",This chapter is about the assessment of journa...,"Calvert, K.R, & Jordan, W.J. (2019). ""Serials ...",paper-title-number-1,http://libres.uncg.edu/ir/wcu/listing.aspx?id=...
1,2018-01-01,Collaborative leadership: cultivating an envir...,Collaborative Librarianship,This paper is about collaboration and leadership.,"Calvert, K.R. (2018). ""Collaborative leadershi...",paper-title-number-2,https://digitalcommons.du.edu/collaborativelib...
2,2015-01-01,Maximizing academic library collections: Measu...,College & Research Libraries,This paper is about the number 3. The number 4...,"Calvert, K.R. (2015). ""Maximizing academic lib...",paper-title-number-3,http://crl.acrl.org/content/76/1/81.full.pdf+html
3,2020-01-01,Closing the Loop on Collections Review,Serials Review,This paper is about the number 3. The number 4...,"Calvert, K. & W.P. Jordan. (2020) Closing the ...",paper-title-number-4,https://doi.org/10.1080/00987913.2020.1806647
4,2013-01-01,Is ILL enough? Examining ILL demand after jour...,Proceedings of the Charleston Library Conference,This paper is about the number 3. The number 4...,"Calvert, K.R, Gee, W., Malliett, J., & Fleming...",paper-title-number-5,http://dx.doi.org/10.5703/1288284315297
5,2013-01-01,Impact of journal cancellations on interlibrar...,Serials Review,This paper is about the number 3. The number 4...,"Fleming, R., Calvert, K., & Hill, K. (2013). I...",paper-title-number-6,http://dx.doi.org/10.1080/00987913.2013.10766391
6,2013-01-01,Starting from scratch on perpetual access,Serials Librarian,This paper is about the number 3. The number 4...,"Calvert, K. (2013). Starting from scratch on p...",paper-title-number-7,http://dx.doi.org/10.1080/0361526X.2013.800464
7,2012-01-01,Giving them what they want: Providing informat...,Proceedings of the Charleston Library Conference,This paper is about the number 3. The number 4...,"Calvert, K., & Fleming, R. (2012). Giving them...",paper-title-number-8,http://dx.doi.org/10.5703/1288284314933


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [19]:
!dir ../_publications/

Invalid switch - "_publications".


In [15]:
!type ../_publications/2009-10-01-paper-title-number-1.md

The syntax of the command is incorrect.
